___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

<h1><p style="text-align: center;">Data Analysis with Python <br>Project - 1</p><h1> - Traffic Police Stops <img src="https://docs.google.com/uc?id=17CPCwi3_VvzcS87TOsh4_U8eExOhL6Ki" class="img-fluid" alt="CLRSWY" width="200" height="100"> 

Does the ``gender`` of a driver have an impact on police behavior during a traffic stop? **In this chapter**, you will explore that question while practicing filtering, grouping, method chaining, Boolean math, string methods, and more!

***

## Examining traffic violations

Before comparing the violations being committed by each gender, you should examine the ``violations`` committed by all drivers to get a baseline understanding of the data.

In this exercise, you'll count the unique values in the ``violation`` column, and then separately express those counts as proportions.

> Before starting your work in this section **repeat the steps which you did in the previos chapter for preparing the data.** Continue to this chapter based on where you were in the end of the previous chapter.

In [132]:
import pandas as pd
import numpy as np
ri = pd.read_csv('police.csv',nrows=50000) 

In [133]:
ri.drop(["county_name", "state","county_fips","fine_grained_location","search_type_raw","location_raw"], inplace = True , axis = 1)

In [134]:
ri.dropna(inplace=True, subset=["driver_gender"], axis=0)

In [135]:
ri.is_arrested=ri.is_arrested.astype("bool")

In [136]:
combined=pd.concat([ri["stop_date"]+" "+ri["stop_time"]])
ri['stop_datetime']=combined
ri.set_index("stop_datetime")


,id,stop_date,stop_time,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,
2005-01-02 01:55,RI-2005-00001,2005-01-02,01:55,600,M,1985.0,20.0,W,White,Speeding,Speeding,False,NaN,False,Citation,False,0-15 Min,False,False,Zone K1
2005-01-02 20:30,RI-2005-00002,2005-01-02,20:30,500,M,1987.0,18.0,W,White,Speeding,Speeding,False,NaN,False,Citation,False,16-30 Min,False,False,Zone X4
2005-01-04 12:55,RI-2005-00004,2005-01-04,12:55,500,M,1986.0,19.0,W,White,Equipment/Inspection Violation,Equipment,False,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-01-06 01:30,RI-2005-00005,2005-01-06,01:30,500,M,1978.0,27.0,B,Black,Equipment/Inspection Violation,Equipment,False,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-01-12 08:05,RI-2005-00006,2005-01-12,08:05,0,M,1973.0,32.0,B,Black,Call for Service,Other,False,NaN,False,Citation,False,30+ Min,True,False,Zone X1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-08-08 22:45,RI-2006-35917,2006-08-08,22:45,300,M,1973.0,33.0,B,Black,Equipment/Inspection Violation,Equipment,False,NaN,False,Citation,False,0-15 Min,False,False,Zone K3
2006-08-08 22:45,RI-2006-35918,2006-08-08,22:45,300,F,1971.0,35.0,B,Black,Speeding,Speeding,False,NaN,False,Citation,False,0-15 Min,True,False,Zone K3
2006-08-08 22:53,RI-2006-35919,2006-08-08,22:53,500,M,1952.0,54.0,W,White,Speeding,Speeding,False,NaN,False,Citation,False,16-30 Min,True,False,Zone X4


**INSTRUCTIONS**

*   Count the unique values in the ``violation`` column, to see what violations are being committed by all drivers.
*   Express the violation counts as proportions of the total.

In [137]:
ri["violation"].nunique()

5

In [138]:
ri.violation.unique()

array(['Speeding', 'Equipment', 'Other', 'Moving violation',
       'Registration/plates'], dtype=object)

In [139]:
ri.violation.value_counts()

Speeding               36111
Moving violation        6522
Equipment               3022
Registration/plates     1463
Other                    892
Name: violation, dtype: int64

In [140]:
ri.violation.value_counts(normalize=True)

Speeding               0.752156
Moving violation       0.135847
Equipment              0.062945
Registration/plates    0.030473
Other                  0.018579
Name: violation, dtype: float64

More than half of all violations are for speeding, followed by other moving violations and equipment violations.

***

## Comparing violations by gender

The question we're trying to answer is whether male and female drivers tend to commit different types of traffic violations.

You'll first create a ``DataFrame`` for each gender, and then analyze the ``violations`` in each ``DataFrame`` separately.

**INSTRUCTIONS**

*   Create a ``DataFrame``, female, that only contains rows in which ``driver_gender`` is ``'F'``.
*   Create a ``DataFrame``, male, that only contains rows in which ``driver_gender`` is ``'M'``.
*   Count the ``violations`` committed by female drivers and express them as proportions.
*   Count the violations committed by male drivers and express them as proportions.

***

In [141]:
female= ri[ri.driver_gender=="F"]

In [142]:
male= ri[ri.driver_gender=="M"]

In [143]:
male.violation.value_counts(normalize=True)

Speeding               0.729518
Moving violation       0.149967
Equipment              0.069595
Registration/plates    0.031584
Other                  0.019337
Name: violation, dtype: float64

In [144]:
female.violation.value_counts(normalize=True)

Speeding               0.811180
Moving violation       0.099031
Equipment              0.045608
Registration/plates    0.027575
Other                  0.016605
Name: violation, dtype: float64

In [145]:
About two-thirds of female traffic stops are for speeding, whereas stops of males are more balanced among the six categories. This doesn't mean that females speed more often than males, however, since we didn't take into account the number of stops or drivers.

SyntaxError: invalid syntax (<ipython-input-145-00cced842442>, line 1)

## Comparing speeding outcomes by gender

When a driver is pulled over for speeding, many people believe that gender has an impact on whether the driver will receive a ticket or a warning. Can you find evidence of this in the dataset?

First, you'll create two ``DataFrames`` of drivers who were stopped for ``speeding``: one containing ***females*** and the other containing ***males***.

Then, for each **gender**, you'll use the ``stop_outcome`` column to calculate what percentage of stops resulted in a ``"Citation"`` (meaning a ticket) versus a ``"Warning"``.

**INSTRUCTIONS**

*   Create a ``DataFrame``, ``female_and_speeding``, that only includes female drivers who were stopped for speeding.
*   Create a ``DataFrame``, ``male_and_speeding``, that only includes male drivers who were stopped for speeding.
*   Count the **stop outcomes** for the female drivers and express them as proportions.
*   Count the **stop outcomes** for the male drivers and express them as proportions.

In [146]:
female_and_speeding=ri[(ri.driver_gender=="F") & (ri.violation=="Speeding")]

In [147]:
male_and_speeding=ri[(ri.driver_gender=="M") & (ri.violation=="Speeding")]

In [148]:
female_and_speeding.stop_outcome.value_counts(normalize=True)

Citation            0.973416
Warning             0.012968
Arrest Driver       0.007410
N/D                 0.003612
Arrest Passenger    0.002316
No Action           0.000278
Name: stop_outcome, dtype: float64

In [149]:
male_and_speeding.stop_outcome.value_counts(normalize=True)

Citation            0.957298
Arrest Driver       0.026230
Warning             0.010666
N/D                 0.003397
Arrest Passenger    0.002015
No Action           0.000395
Name: stop_outcome, dtype: float64

* Interesting! The numbers are similar for males and females: about 95% of stops for speeding result in a ticket. Thus, the data fails to show that gender has an impact on who gets a ticket for speeding.

***

## Calculating the search rate

During a traffic stop, the police officer sometimes conducts a search of the vehicle. In this exercise, you'll calculate the percentage of all stops that result in a vehicle search, also known as the **search rate**.

**INSTRUCTIONS**

*   Check the data type of ``search_conducted`` to confirm that it's a ``Boolean Series``.
*   Calculate the search rate by counting the ``Series`` values and expressing them as proportions.
*   Calculate the search rate by taking the mean of the ``Series``. (It should match the proportion of ``True`` values calculated above.)

In [151]:
ri.search_conducted.dtype #ri[["search_conducted"]].info()

dtype('bool')

***

In [152]:
ri.search_conducted.value_counts(normalize=True)

False    0.958092
True     0.041908
Name: search_conducted, dtype: float64

In [153]:
(ri.search_conducted.mean()).round(6)


0.041908

## Comparing search rates by gender

You'll compare the rates at which **female** and **male** drivers are searched during a traffic stop. Remember that the vehicle search rate across all stops is about **3.8%**.

First, you'll filter the ``DataFrame`` by gender and calculate the search rate for each group separately. Then, you'll perform the same calculation for both genders at once using a ``.groupby()``.

**INSTRUCTIONS 1/3**

*   Filter the ``DataFrame`` to only include **female** drivers, and then calculate the search rate by taking the mean of ``search_conducted``.

In [154]:
ri[ri.driver_gender=="F"].search_conducted.mean().round(6)

0.017807

**INSTRUCTIONS 2/3**

*   Filter the ``DataFrame`` to only include **male** drivers, and then repeat the search rate calculation.

In [155]:
ri[ri.driver_gender=="M"].search_conducted.mean()#ri[ri["driver_gender"]=="M"].search_conducted.mean()

0.05115126365234431

**INSTRUCTIONS 3/3**

*   Group by driver gender to calculate the search rate for both groups simultaneously. (It should match the previous results.)

In [156]:
ri.groupby(by="driver_gender").search_conducted.mean()

driver_gender
F    0.017807
M    0.051151
Name: search_conducted, dtype: float64

* Male drivers are searched more than twice as often as female drivers.

***

## Adding a second factor to the analysis

Even though the search rate for males is much higher than for females, it's possible that the difference is mostly due to a second factor.

For example, you might hypothesize that the search rate varies by violation type, and the difference in search rate between males and females is because they tend to commit different violations.

You can test this hypothesis by examining the search rate for each combination of gender and violation. If the hypothesis was true, you would find that males and females are searched at about the same rate for each violation. Find out below if that's the case!

**INSTRUCTIONS 1/2**

*   Use a ``.groupby()`` to calculate the search rate for each combination of gender and violation. Are males and females searched at about the same rate for each violation?

In [157]:
ri.groupby(["driver_gender","violation"]).search_conducted.mean()

driver_gender  violation          
F              Equipment              0.079077
               Moving violation       0.047800
               Other                  0.045249
               Registration/plates    0.114441
               Speeding               0.006854
M              Equipment              0.123395
               Moving violation       0.088778
               Other                  0.154993
               Registration/plates    0.171533
               Speeding               0.028560
Name: search_conducted, dtype: float64

**INSTRUCTIONS 2/2**

*   Reverse the ordering to group by violation before gender. The results may be easier to compare when presented this way.

In [158]:
ri.groupby(["violation","driver_gender"]).search_conducted.mean()

violation            driver_gender
Equipment            F                0.079077
                     M                0.123395
Moving violation     F                0.047800
                     M                0.088778
Other                F                0.045249
                     M                0.154993
Registration/plates  F                0.114441
                     M                0.171533
Speeding             F                0.006854
                     M                0.028560
Name: search_conducted, dtype: float64

* For all types of violations, the search rate is higher for males than for females.disproving our hypothesis.

***

## Counting protective frisks

During a vehicle search, the police officer may pat down the driver to check if they have a weapon. This is known as a ``"protective frisk."``

You'll first check to see how many times "Protective Frisk" was the only search type. Then, you'll use a string method to locate all instances in which the driver was frisked.

**INSTRUCTIONS**

*   Count the ``search_type`` values to see how many times ``"Protective Frisk"`` was the only search type.
*   Create a new column, frisk, that is ``True`` if ``search_type`` contains the string ``"Protective Frisk"`` and ``False`` otherwise.
*   Check the data type of frisk to confirm that it's a ``Boolean Series``.
*   Take the sum of frisk to count the total number of frisks.

In [190]:
ri.search_type.value_counts(dropna=False) 

NaN                                                         45998
Incident to Arrest                                            958
Probable Cause                                                244
Protective Frisk                                              204
Inventory                                                     117
Incident to Arrest,Inventory                                  116
Incident to Arrest,Probable Cause                              76
Incident to Arrest,Protective Frisk                            63
Reasonable Suspicion                                           43
Probable Cause,Protective Frisk                                36
Incident to Arrest,Inventory,Protective Frisk                  33
Inventory,Protective Frisk                                     23
Incident to Arrest,Probable Cause,Protective Frisk             20
Incident to Arrest,Inventory,Probable Cause                    19
Protective Frisk,Reasonable Suspicion                          16
Inventory,

In [172]:
ri["frisk"]=ri.search_type.str.contains("Frisk",na=False)

In [177]:
ri.frisk.dtype

dtype('bool')

In [176]:
ri.frisk.sum() #ri[["frisk"]].value_counts()

403

***

## Comparing frisk rates by gender

You'll compare the rates at which female and male drivers are frisked during a search. Are males frisked more often than females, perhaps because police officers consider them to be higher risk?

Before doing any calculations, it's important to filter the ``DataFrame`` to only include the relevant subset of data, namely stops in which a search was conducted.

**INSTRUCTIONS**

*   Create a ``DataFrame``, searched, that only contains rows in which ``search_conducted`` is ``True``.
*   Take the mean of the frisk column to find out what percentage of searches included a frisk.
*   Calculate the frisk rate for each gender using a ``.groupby()``.

In [191]:
searched = ri[ri.search_conducted == True]

In [195]:
searched.frisk.mean() # ri.frisk.mean()=0.00839408456571547 includes all searchs

0.20029821073558648

In [197]:
searched.groupby(by="driver_gender").frisk.mean()

driver_gender
F    0.164557
M    0.205070
Name: frisk, dtype: float64